In [1]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
path_root = '/home/dmr/DatasetForExperiments/adware'

In [26]:
from keras.preprocessing.image import ImageDataGenerator
batches = ImageDataGenerator().flow_from_directory(directory=path_root, target_size=(64,64), batch_size=10000)

Found 107 images belonging to 10 classes.


In [3]:
# Print information about Images 
imgs, labels = next(batches)
imgs.shape # Images with width x length x depth
labels.shape # Lablels with batch_size, number of classes

(107, 10)

In [4]:
import numpy as np
import scipy as sp
from PIL import Image

# Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs/255.,labels, test_size=0.3)

In [5]:
X_train.shape

(74, 64, 64, 3)

In [6]:
X_test.shape

(33, 64, 64, 3)

In [7]:
y_train.shape

(74, 10)

In [8]:
y_test.shape

(33, 10)

In [9]:
# CNN Model

In [10]:
import keras
import tensorflow
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import AUC

In [11]:
num_classes = 10

In [12]:
def malware_model():
    Malware_model = Sequential()
    # Convolutional Layer : 30 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(30, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3)))

    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #Convolutional Layer : 15 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(16, (3, 3), activation='relu'))
    
    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # DropOut Layer : Dropping 25% of neurons.
    Malware_model.add(Dropout(0.25))
    
    # Flatten Layer
    Malware_model.add(Flatten())
    
    # Dense/Fully Connected Layer : 128 Neurons, Relu activation function
    Malware_model.add(Dense(128, activation='relu'))
    
   # Malware_model.add(Dense(64, activation='relu'))
    
    # DropOut Layer : Dropping 50% of neurons.
    Malware_model.add(Dropout(0.5))
    
    # Dense/Fully Connected Layer : 50 Neurons, Softmax activation function
    Malware_model.add(Dense(50, activation='relu'))
    
    # Dense/Fully Connected Layer : num_class Neurons, Softmax activation function
    Malware_model.add(Dense(num_classes, activation='softmax'))
    
    
    Malware_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return Malware_model


    # sparse_categorical_cross_entropy
    # categorical_crossentropy
    

In [13]:
from keras import backend as K

def recall_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_test, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_test, y_pred):
    precision = precision_m(y_test, y_pred)
    recall = recall_m(y_test, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [25]:
Malware_model = malware_model()

In [15]:
Malware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 30)        840       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 16)        4336      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [16]:
y_train.shape

(74, 10)

In [17]:
y_train_new = np.argmax(y_train, axis=1)

In [18]:
y_train_new

array([2, 2, 5, 8, 3, 5, 2, 0, 1, 2, 6, 4, 7, 4, 9, 4, 4, 0, 7, 3, 5, 9,
       3, 1, 5, 6, 3, 4, 8, 7, 2, 4, 7, 5, 1, 7, 4, 5, 7, 2, 0, 6, 9, 6,
       6, 1, 1, 3, 3, 0, 1, 4, 3, 9, 8, 6, 2, 5, 6, 3, 9, 0, 0, 6, 5, 3,
       4, 0, 1, 1, 9, 2, 2, 5])

In [19]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(y_train_new),
                                                 y = y_train_new)

class_weights = {l:c for l,c in zip(np.unique(y_train_new), class_weights)}

In [20]:
Malware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_m,precision_m, recall_m,AUC()])

In [21]:
Malware_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,  class_weight=class_weights)

2023-05-02 19:00:15.737473: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/100
3/3 [==============================] - 1s 249ms/step - loss: 0.6938 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.4950 - val_loss: 0.5608 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5317
Epoch 2/100
3/3 [==============================] - 0s 42ms/step - loss: 0.5781 - f1_m: 0.0256 - precision_m: 0.0556 - recall_m: 0.0167 - auc: 0.5286 - val_loss: 0.4280 - val_f1_m: 0.0294 - val_precision_m: 0.2500 - val_recall_m: 0.0156 - val_auc: 0.5604
Epoch 3/100
3/3 [==============================] - 0s 39ms/step - loss: 0.4768 - f1_m: 0.0400 - precision_m: 0.1146 - recall_m: 0.0258 - auc: 0.5380 - val_loss: 0.3866 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5583
Epoch 4/100
3/3 [==============================] - 0s 41ms/step - loss: 0.4548 - f1_m: 0.0427 - precision_m: 0.1227 - recall_m: 0.0260 - auc: 0.5208 - val_loss: 0.3570 - val_f1_m: 0.0000e+00 - val_prec

3/3 [==============================] - 0s 42ms/step - loss: 0.2339 - f1_m: 0.3061 - precision_m: 1.0000 - recall_m: 0.1820 - auc: 0.8980 - val_loss: 0.2801 - val_f1_m: 0.0588 - val_precision_m: 0.5000 - val_recall_m: 0.0312 - val_auc: 0.7830
Epoch 32/100
3/3 [==============================] - 0s 45ms/step - loss: 0.2183 - f1_m: 0.3770 - precision_m: 0.8681 - recall_m: 0.2432 - auc: 0.9279 - val_loss: 0.2785 - val_f1_m: 0.0303 - val_precision_m: 0.5000 - val_recall_m: 0.0156 - val_auc: 0.7919
Epoch 33/100
3/3 [==============================] - 0s 42ms/step - loss: 0.2120 - f1_m: 0.4683 - precision_m: 0.9167 - recall_m: 0.3177 - auc: 0.9183 - val_loss: 0.2751 - val_f1_m: 0.0278 - val_precision_m: 0.1250 - val_recall_m: 0.0156 - val_auc: 0.8064
Epoch 34/100
3/3 [==============================] - 0s 41ms/step - loss: 0.2231 - f1_m: 0.3920 - precision_m: 0.7034 - recall_m: 0.2742 - auc: 0.9013 - val_loss: 0.2696 - val_f1_m: 0.1026 - val_precision_m: 0.2857 - val_recall_m: 0.0625 - val_auc: 

3/3 [==============================] - 0s 46ms/step - loss: 0.0821 - f1_m: 0.8565 - precision_m: 0.9216 - recall_m: 0.8023 - auc: 0.9930 - val_loss: 0.3056 - val_f1_m: 0.6538 - val_precision_m: 0.7000 - val_recall_m: 0.6250 - val_auc: 0.8047
Epoch 64/100
3/3 [==============================] - 0s 44ms/step - loss: 0.0817 - f1_m: 0.8072 - precision_m: 0.8495 - recall_m: 0.7690 - auc: 0.9911 - val_loss: 0.2986 - val_f1_m: 0.6667 - val_precision_m: 0.7045 - val_recall_m: 0.6406 - val_auc: 0.8128
Epoch 65/100
3/3 [==============================] - 0s 46ms/step - loss: 0.0746 - f1_m: 0.8796 - precision_m: 0.8999 - recall_m: 0.8607 - auc: 0.9954 - val_loss: 0.3024 - val_f1_m: 0.6373 - val_precision_m: 0.6842 - val_recall_m: 0.6094 - val_auc: 0.8196
Epoch 66/100
3/3 [==============================] - 0s 47ms/step - loss: 0.0760 - f1_m: 0.8904 - precision_m: 0.9102 - recall_m: 0.8716 - auc: 0.9923 - val_loss: 0.3089 - val_f1_m: 0.6765 - val_precision_m: 0.7368 - val_recall_m: 0.6406 - val_auc: 

3/3 [==============================] - 0s 43ms/step - loss: 0.0573 - f1_m: 0.8554 - precision_m: 0.8592 - recall_m: 0.8518 - auc: 0.9946 - val_loss: 0.3367 - val_f1_m: 0.6833 - val_precision_m: 0.6964 - val_recall_m: 0.6719 - val_auc: 0.8226
Epoch 96/100
3/3 [==============================] - 0s 43ms/step - loss: 0.0393 - f1_m: 0.9097 - precision_m: 0.9140 - recall_m: 0.9055 - auc: 0.9984 - val_loss: 0.3317 - val_f1_m: 0.6930 - val_precision_m: 0.7200 - val_recall_m: 0.6719 - val_auc: 0.8230
Epoch 97/100
3/3 [==============================] - 0s 40ms/step - loss: 0.0414 - f1_m: 0.9648 - precision_m: 0.9648 - recall_m: 0.9648 - auc: 0.9985 - val_loss: 0.3499 - val_f1_m: 0.6930 - val_precision_m: 0.7200 - val_recall_m: 0.6719 - val_auc: 0.7939
Epoch 98/100
3/3 [==============================] - 0s 37ms/step - loss: 0.0416 - f1_m: 0.9187 - precision_m: 0.9231 - recall_m: 0.9143 - auc: 0.9977 - val_loss: 0.3678 - val_f1_m: 0.6930 - val_precision_m: 0.7200 - val_recall_m: 0.6719 - val_auc: 

In [22]:
scores = Malware_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.3714 - f1_m: 0.7069 - precision_m: 0.7308 - recall_m: 0.6875 - auc: 0.8093


In [23]:
loss, accuracy, f1_score, precision, recall = Malware_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.3714 - f1_m: 0.7069 - precision_m: 0.7308 - recall_m: 0.6875 - auc: 0.8093


In [24]:
print('Final CNN accuracy: ', scores[1])

Final CNN accuracy:  0.7068964242935181
